In [12]:
#Most of these will be removed before the final submission. They are holdovers from the code I used to make this.
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

from graphviz import Source
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn.neural_network import MLPClassifier
from sklearn import neural_network
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

import math
import csv
import os

In [19]:
#Get the current path where the code is saved
path = os.getcwd()

#Change working directory for Python to the location where the code is saved
os.chdir(path)

#Read the excel file with data. Make sure it is saved in the same location as the code!!
data = pd.read_excel("Airbnb_Open_Data.xlsx")

#Give a description of the data that was loaded
print("\nHead:\n",data.head())
print("\nDescription:\n",data.price.describe())

#Create a dataframe with the new data,
frame=DataFrame(data)

#Assign columns 2-18 as variable X. X will represent the independent variables we will use to make predictions.
X = data.iloc[:,1:17]

In [20]:
#If un-commented, this statement can be used to show how many columns are in X.
#print("\n\nX: \n",len(X.iloc[0,:]))



X: 
 16


In [22]:
#Assign the "price" variable to y. y will represent the dependent variable for which we are solving.
y = X.price

#Remove columns that will not be used in this analysis. We need to drop "price" and anything having to do with reviews. 
X = X.drop(columns=['last review', 'reviews per month', 'number of reviews', 'review rate number', 'price'])

#Create dummy variables for columns with character data types
X = pd.get_dummies(X, columns=['host_identity_verified', 'neighbourhood group', 'neighbourhood', 'instant_bookable', 'cancellation_policy', 'room type'])

#Show details about the dataset after adding dummy variables
X.head()

,Construction year,service fee,minimum nights,calculated host listings count,availability 365,host_identity_verified_unconfirmed,host_identity_verified_verified,neighbourhood group_Bronx,neighbourhood group_Brooklyn,neighbourhood group_Manhattan,...,neighbourhood_Woodside,instant_bookable_False,instant_bookable_True,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,room type_Entire home/apt,room type_Hotel room,room type_Private room,room type_Shared room
0,2020.0,193.0,10.0,6.0,286.0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
1,2007.0,28.0,30.0,2.0,228.0,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,0
2,2005.0,124.0,3.0,1.0,352.0,1,0,0,0,1,...,0,0,1,1,0,0,0,0,1,0
3,2009.0,41.0,10.0,1.0,289.0,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,0
4,2013.0,115.0,3.0,1.0,374.0,0,1,0,0,1,...,0,0,1,1,0,0,1,0,0,0


In [23]:
#Split the data into test and training data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=34)

In [24]:
#If un-commented, these statements can be used to confirm that the X and y sets created are the correct length. This is useful in troubleshooting.
#print(len(X_train))
#print(len(X_train))
#print(len(X_test))
#print(len(y_test))

69800
69800
29915
29915


In [13]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
    
# Initialize Lasso model
lasso = Lasso(random_state=42)

# Define a range of alpha values to search
param_grid = {'alpha': np.logspace(-4, 0, 50)}

# Use GridSearchCV to find the optimal alpha using cross-validation
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best Lasso model
best_lasso = grid_search.best_estimator_
print(f"Best alpha: {best_lasso.alpha}")

# Access the coefficients
coefficients = best_lasso.coef_
feature_names = X.columns

# Identify selected features (non-zero coefficients)
selected_features = feature_names[coefficients != 0]
discarded_features = feature_names[coefficients == 0]

#Print details about the features and coefficients that were selected
print(f"\n\nSelected Features: {list(selected_features)}")
#print(f"\n\nDiscarded Features: {list(discarded_features)}")
print(f"\n\nCoefficients: {dict(zip(feature_names, coefficients))}")

# Evaluate the model
y_pred = best_lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"\nMean Squared Error on Test Set: {mse}")

Best alpha: 0.009102981779915217


Selected Features: ['Construction year', 'service fee', 'neighbourhood_Allerton', 'neighbourhood_Arverne', 'neighbourhood_Belmont', 'neighbourhood_Borough Park', 'neighbourhood_Breezy Point', 'neighbourhood_Bronxdale', 'neighbourhood_Cambria Heights', 'neighbourhood_Co-op City', 'neighbourhood_Concord', 'neighbourhood_Dyker Heights', 'neighbourhood_East Flatbush', 'neighbourhood_East Village', 'neighbourhood_Emerson Hill', 'neighbourhood_Fordham', 'neighbourhood_Fort Greene', 'neighbourhood_Glendale', 'neighbourhood_Gowanus', 'neighbourhood_Hollis', 'neighbourhood_Huguenot', 'neighbourhood_Jackson Heights', 'neighbourhood_Kips Bay', 'neighbourhood_Little Neck', 'neighbourhood_Manhattan Beach', 'neighbourhood_Oakwood', 'neighbourhood_Parkchester', 'neighbourhood_Port Morris', 'neighbourhood_Queens Village', 'neighbourhood_Rego Park', 'neighbourhood_Roosevelt Island', 'neighbourhood_Rossville', 'neighbourhood_Soundview', 'neighbourhood_South Beach', 'ne

In [14]:
#Create a linear regression model for the data
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
#Determine accuracies for the predictions
accuracies = cross_val_score(lr, X_train, y_train, cv=10)
print("\n\nAccuracies:\n", accuracies)



Accuracies:
 [0.99998174 0.99998159 0.99998153 0.99998189 0.9999818  0.99998166
 0.99998163 0.99998169 0.99998192 0.9999817 ]


In [16]:
#Fit the linear regression to the training model
lr.fit(X_train, y_train)

#Print the coefficients and intercept
print("\nLR Coef:",lr.coef_)
#print("\n\nMAX", lr.coef_.max())
print("\nLR Intercept:",lr.intercept_)

#Use the model to make predictions on the test and training sets
lr_train_pred = lr.predict(X_train)
lr_test_pred = lr.predict(X_test)


LR Coef: [-1.83550748e-03  4.99965423e+00 -5.09989783e-05 -1.67262380e-04
 -2.00081720e-05  4.42663086e-05 -4.42663086e-05 -5.06506363e-02
 -4.11115277e-02  7.52911810e-03  3.48908778e-03  2.42103533e-02
  5.65336049e-02 -3.68264978e-01  9.84146515e-02  1.17450262e-01
  2.60762526e-01 -1.73428909e-02 -1.57001370e-01 -5.07660573e-02
 -4.05220310e-02 -3.74249568e-02 -5.44758161e-01 -1.41007231e-01
  3.01444525e-02 -1.72090309e-01  1.03283086e-02 -9.85802813e-02
 -6.44218223e-01  5.86718813e-01  1.24323372e-01  1.26233809e-01
  4.79336480e-02 -2.07991899e-01  1.09128149e+00  1.01532625e-01
 -3.63501265e-02 -7.94188815e-01  1.35129588e-01 -2.20013706e-02
  9.17534878e-01  1.70513316e-02  3.81656166e-01  1.70843896e-02
  1.42774114e-02  3.60864556e-01  2.08491309e-01 -8.94333792e-02
  1.98077560e+00 -1.16738228e-01  1.81642286e-01 -7.41475098e-02
 -3.17618421e-01 -3.85999445e-02  2.53497514e-01  2.99677737e-02
 -1.22969022e+00 -4.36088297e-02  2.68915645e-01  1.73102281e-01
  3.41788051e-0

In [17]:
#Use cross-validation to determine the accuracy of the linear regression
scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(lr, X_train, y_train, scoring='r2', cv=10, return_train_score = True)
scores_frame=pd.DataFrame(scores)
#print("\n\nScores:\n",scores_frame)

#Print metrics for the test and training sets
print(metrics.r2_score(y_train, lr_train_pred))
print(metrics.mean_squared_error(y_train, lr_train_pred))
print(metrics.r2_score(y_test, lr_test_pred))
print(metrics.mean_squared_error(y_test, lr_test_pred))

0.9999818507975274
1.9993171587773721
0.9999817212275803
2.0016876760806035


In [18]:
#Create a decision tree model
RT = tree.DecisionTreeRegressor(min_samples_leaf=5, random_state=39)

#Fit the model to the training data
RT.fit(X_train, y_train)
RT_pred_train = RT.predict(X_train)
RT_pred_test = RT.predict(X_test)
print("\nRegression Tree Training Set Mean Squared Error:",metrics.mean_squared_error(y_train, RT_pred_train))
print("\nRegression Tree Training Set R2:\n",metrics.r2_score(y_train, RT_pred_train))
print("\nRegression Tree Test Set Mean Squared Error:",metrics.mean_squared_error(y_test, RT_pred_test))
print("\nRegression Tree Test Set R2:\n",metrics.r2_score(y_test, RT_pred_test))

RT.feature_importances_
pd.DataFrame({'variable':X_final.columns[:247], 'importance':RT.feature_importances_})
dot_data = tree.export_graphviz(RT, out_file=None, feature_names = X_train.columns)
Source(dot_data).render('Regression_Tree')

scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(RT, X_train, y_train, scoring='r2', cv=10, return_train_score = True)
scores_frame=pd.DataFrame(scores)
print("\n\nScores:\n",scores_frame)



Regression Tree Training Set Mean Squared Error: 0.8826185581769392

Regression Tree Training Set R2:
 0.9999919878530287

Regression Tree Test Set Mean Squared Error: 2.4180215899124122

Regression Tree Test Set R2:
 0.9999779193992769


Scores:
    fit_time  score_time  test_score  train_score
0  0.951659    0.008801    0.999977     0.999992
1  0.879055    0.006331    0.999977     0.999992
2  0.887552    0.006376    0.999978     0.999992
3  0.889791    0.006389    0.999978     0.999992
4  0.883221    0.006214    0.999978     0.999992
5  0.879890    0.006327    0.999977     0.999992
6  0.878751    0.006287    0.999977     0.999992
7  0.959265    0.006234    0.999978     0.999992
8  0.899534    0.006300    0.999977     0.999992
9  0.891109    0.006349    0.999978     0.999992
